In [40]:
import database
import req
import pprint
import json
import os
import csv

dbsavefile = '/DBsave.json'
#DB = database.DB_init()
DB=loadDB()
pwd = os.getcwd()
pp = pprint.PrettyPrinter(indent=4)
pprint = pp.pprint
Teams = DB['Teams']
Games = DB['Games']



In [41]:

def saveDB(db):
    with open("".join([pwd,dbsavefile]), 'wb') as jsonfile: 
            json.dump(db,jsonfile)

def num(s):
    try:
        return int(s)
    except ValueError:
        try:
            return float(s)
        except ValueError:
            return s

def d_or_v(val):
    if isinstance(val,dict):
        return key_to_int(val)
    else:
        return val
   
def key_to_int(d):
    return {num(k):d_or_v(v) for k,v in d.iteritems()}   

def loadDB():
    with open("".join([pwd,dbsavefile]), 'rb') as jsonfile: 
        db = json.load(jsonfile)
    return key_to_int(db)

    
def print_team(d,score = 0.0):
    pprint(''.join([str(d['id']),': ',d['name'],'  strength = ',str(score)]))

def print_game(d):
    if d['status']==1:
        pprint("".join([str(d['id']),':', 'r', str(d['round']),'  ',
                        str(d['scores'][d['team_1']]),' -- ',
                        DB['Teams'][d['team_1']]['name'],
                        ' vs ', DB['Teams'][d['team_2']]['name'], ' -- ',
                        str(d['scores'][d['team_2']])]))
    else:
        pprint("".join([str(d['id']),':','r',str(d['round']),'  ',
                DB['Teams'][d['team_1']]['name'],
                ' vs ', DB['Teams'][d['team_2']]['name']]))
def list_teams(session_id):
    session = DB['Sessions'][session_id]
    for id in session['teams']:
        s = session['strengths'][id]
        print_team(DB['Teams'][id],score = s)
def list_games(session_id):
    session = DB['Sessions'][session_id]
    for id in session['games']:
        print_game(DB['Games'][id])

def export_results(session_id):
    session = DB['Sessions'][session_id]
    d = [[id,DB['Teams'][id]['name'],session['strengths'][id]] for id in session['teams']]

    with open("".join([pwd,'/results.csv']), 'wb') as csvfile:

        writer = csv.writer(csvfile,delimiter='\t')
    
        for line in d:
            writer.writerow(line)

            
def export_games(session_id):
    session = DB['Sessions'][session_id]
    played_games = [Games[id] for id in session['games'] if Games[id]['status'] == 1]
    unplayed_games = [Games[id] for id in session['games'] if not Games[id]['status'] == 1]

    pd = [[g['id'],Teams[g['team_1']]['name'],Teams[g['team_2']]['name'],
            g['round'],g['scores'][g['team_1']],
            g['scores'][g['team_2']]] for g in played_games]    
    upd = [[g['id'],Teams[g['team_1']]['name'],Teams[g['team_2']]['name'],
            g['round'],'none',
            'none'] for g in unplayed_games]    
    with open("".join([pwd,'/games.csv']), 'wb') as csvfile:

        writer = csv.writer(csvfile,delimiter='\t')
    
        for line in pd:
           writer.writerow(line)
        
        for line in upd:
           writer.writerow(line)
        
def export_suggested_games(session_id):
    session = DB['Sessions'][session_id]
    suggested_games = session['suggestedGames'].values()

    pd = [[g['id'],Teams[g['team_1']]['name'],Teams[g['team_2']]['name'],
            g['round']] for g in suggested_games]    

    with open("".join([pwd,'/suggested_games.csv']), 'wb') as csvfile:

        writer = csv.writer(csvfile,delimiter='\t')
    
        for line in pd:
           writer.writerow(line)
        
def export_teams_list(session_id):
    session = DB['Sessions'][session_id]
    teams = [DB['Teams'][id] for id in session['teams']]

    pd = [[t['id'],t['name']] for t in teams]    

    with open("".join([pwd,'/team_list.csv']),'wb') as csvfile:

        writer = csv.writer(csvfile,delimiter='\t')
    
        for line in pd:
           writer.writerow(line)
        




In [42]:
sess_id = 1
export_teams_list(sess_id)

def compute_strengths():
    req.update_strengths(DB,sess_id)
    export_results(sess_id)

def stage_round(round = 0):
    req.get_suggested_games(DB,sess_id,round)
    export_suggested_games(sess_id)
    print('done')
    
def update_score(game_id=-1,score_1 = -1, score_2 = -1):
    req.update_score(DB,game_id,score_1,score_2)
    export_games(sess_id)
    print_game(Games[game_id])

def add_suggested_games():
    req.add_suggested_games(DB,sess_id)
    export_games(sess_id)
def add_suggested_game(game_id=0):
    req.add_game(DB,sess_id,DB['Sessions'][sess_id]['suggestedGames'][game_id])
    export_games(sess_id)

def add_game(team_1 = 0, team_2 = 0, round = 0):
    req.add_game(DB,sess_id,req.new_game(team_1,team_2,round))
    export_games(sess_id)

def remove_game(game_id=-1):
    req.remove_game(DB, sess_id, game_id)
    export_games(sess_id)


In [184]:
#Add a game to the session. Use the team ID to specify the teams
add_game(team_1 = 26, team_2 = 27, round = 1 )

In [157]:
# update the scores in a game. get the game id from the list of games
update_score(game_id=2,score_1 = 1, score_2 = 1)

'2:r1  1 -- JKL vs The Inseminators -- 1'


In [158]:
# remove a game from the session
remove_game(game_id=9)

In [37]:
#Compute the strengths of the teams. This may take some time.
compute_strengths()

IndexError: too many indices for array

In [6]:
#Suggest games for the next round. This may take quite some time.
stage_round(round = 2)

IndexError: too many indices for array

In [163]:
#This will add the suggested games to the list of games
add_suggested_games()

In [165]:
#add one of the suggested games to the list of games
add_suggested_game(game_id=4)